# load libraries and data directory

In [64]:
suppressMessages(library(oligo)) 
suppressMessages(library(RankProd)) 
suppressMessages(library(limma))
library(gplots)
library(plotrix)
library(RColorBrewer)
library(methods)
library(edgeR)
library(ggbiplot)
library(genefilter)

suppressMessages(library(gtools)) 
suppressMessages(library(Hmisc))
suppressMessages(library(minet))
suppressMessages(library(reshape2))
suppressMessages(library(plyr))
suppressMessages(library(outliers))
suppressMessages(library(magrittr))
suppressMessages(library(dplyr))
suppressMessages(library(tidyr))
suppressMessages(library(purrr))
suppressMessages(library(stringr))

library(tibble)
library(readr)

options(stringsAsFactors = FALSE)

In [72]:
mRNA_data_dir = '/home/guillaume/data/GSK_enfisema/arrays_mRNA/'
pheno_csv = '/home/guillaume/data/GSK_enfisema/pheno_groups.csv'
working_dir = '/home/guillaume/Documents/GSK_enfisema/'

# read U219 array cell files and RMA normalize

## create phenotypic data_frame

In [73]:
groups = read.csv(pheno_csv,header = TRUE)

fi = list.files(mRNA_data_dir)[grepl('.CEL',list.files(mRNA_data_dir))]

pheno = data_frame(sample = fi) %>% rowwise %>% 
  mutate(patient = strsplit(sample,split='_')[[1]][1],      
         pat = gsub('RF-','',patient),
         pat = gsub('RF','',pat),
         pat = gsub('-1','',pat),
         pat = gsub('-2','',pat),
         pat = gsub('-3','',pat)) %>% ungroup %>% select(-patient)

#pheno$pat[pheno$pat %in% groups$pat]
#pheno$pat[!pheno$pat %in% groups$pat]
#groups$pat[!groups$pat %in% pheno$pat]

pheno %<>% left_join(groups, by='pat') 
write_csv(pheno, str_c(working_dir,'pheno.csv'))

In [74]:
pheno %>% dim

[1] 94 18

## read CEL files

In [76]:
ph <- new("AnnotatedDataFrame", data=pheno)
pheno = as.data.frame(pData(ph))
affyExonFS = read.celfiles(str_c(mRNA_data_dir,ph$sample), 
                             phenoData=ph, sampleNames=ph$pat, verbose=F)

Reading in : /home/guillaume/data/GSK_enfisema/arrays_mRNA/101_G03.CEL
Reading in : /home/guillaume/data/GSK_enfisema/arrays_mRNA/102_G04.CEL
Reading in : /home/guillaume/data/GSK_enfisema/arrays_mRNA/103_G05.CEL
Reading in : /home/guillaume/data/GSK_enfisema/arrays_mRNA/104_G06.CEL
Reading in : /home/guillaume/data/GSK_enfisema/arrays_mRNA/106_T-32_(106)_G08.CEL
Reading in : /home/guillaume/data/GSK_enfisema/arrays_mRNA/107_G09.CEL
Reading in : /home/guillaume/data/GSK_enfisema/arrays_mRNA/108_T-42_(108)_G10.CEL
Reading in : /home/guillaume/data/GSK_enfisema/arrays_mRNA/109_T-31_(109)_G11.CEL
Reading in : /home/guillaume/data/GSK_enfisema/arrays_mRNA/10_A09.CEL
Reading in : /home/guillaume/data/GSK_enfisema/arrays_mRNA/110_T-30_(110)_G12.CEL
Reading in : /home/guillaume/data/GSK_enfisema/arrays_mRNA/111_H01.CEL
Reading in : /home/guillaume/data/GSK_enfisema/arrays_mRNA/112_H02.CEL
Reading in : /home/guillaume/data/GSK_enfisema/arrays_mRNA/113_H03.CEL
Reading in : /home/guillaume/data/

Warning message:
In read.celfiles(str_c(mRNA_data_dir, ph$sample), phenoData = ph, : 'channel' automatically added to varMetadata in phenoData.